In [1]:
from fastai.data.all import *
from modules import FastSpeech
from data import TTSDataset, collate_fn

In [10]:
path_ds = Path("../data/LJSpeech-1.1/")
path_vocab = Path("../data/CMUDict/cmudict-0.7b.symbols.txt")

sample_rate = 22050
n_fft = 2048
hop_length = 512 
n_bins = 80 

ratio = 0.1
bs = 32
epochs = 10
lr = 0.0001

n_hidden = 32
n_heads = 8
n_blocks = 6
upsample_ratio = 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
ds = TTSDataset(path_ds, path_vocab, sample_rate, n_fft, hop_length, n_bins, ratio)
dl = torch.utils.data.DataLoader(ds, bs, shuffle=True,
                                 collate_fn=partial(collate_fn, pad_num=ds.pad_num))

In [12]:
mels, phones, durations = first(dl)
L(map(lambda x: x.shape, (mels, phones, durations)))

(#3) [torch.Size([32, 80, 434]),torch.Size([32, 115]),torch.Size([32, 115])]

In [13]:
model = FastSpeech(len(ds.vocab), n_hidden, n_bins, n_heads, n_blocks).to(device)

In [14]:
out = model(phones, durations, upsample_ratio)
out.shape

KeyboardInterrupt: 

In [15]:
optim = torch.optim.Adam(model.parameters(), lr)
loss_fn = F.mse_loss

In [16]:
loss_fn(out, mels)

NameError: name 'out' is not defined

In [17]:
from tqdm.auto import tqdm

In [19]:
loss_tracking = []
for i in tqdm(range(epochs)):
    running_loss = 0
    for mels, phones, durations in tqdm(dl):
        mels, phones, durations = map(lambda x: x.to(device), (mels, phones, durations))
        optim.zero_grad()
        pred = model(phones, durations, upsample_ratio)
        loss = loss_fn(pred, mels)
        loss.backward()
        optim.step()
        
        running_loss += loss.detach().item() / len(dl)
    loss_tracking.append(running_loss)
    print(f"Epoch {i}, Loss: {running_loss}")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/41 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
out = model(phones[None, 0], durations[None, 0], upsample_ratio).detach()

In [ ]:
import librosa

In [ ]:
plt.imshow(librosa.power_to_db(out[0].numpy()), origin="lower");